In [1]:
!curl https://temp56478.blob.core.windows.net/sefiyat/ecommerce.csv -o ecommerce.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9095k  100 9095k    0     0  32.5M      0 --:--:-- --:--:-- --:--:-- 32.4M


In [2]:
import pandas as pd

df = pd.read_csv('ecommerce.csv')
df.head()

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (7,8,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Order ID,Order Date,Ship Date,Aging,Ship Mode,Product Category,Product,Sales,Quantity,Discount,...,Shipping Cost,Order Priority,Customer ID,Customer Name,Segment,City,State,Country,Region,Months
0,AU-2015-1,11/9/2015,11/17/2015,8.0,First Class,Auto & Accessories,Car Media Players,140.00,2,0.05,...,4.60,Medium,LS-001,Lane Daniels,Consumer,Brisbane,Queensland,Australia,Oceania,Nov
1,AU-2015-2,6/30/2015,7/2/2015,2.0,First Class,Auto & Accessories,Car Speakers,211.00,3,0.03,...,11.20,Medium,IZ-002,Alvarado Kriz,Home Office,Berlin,Berlin,Germany,Central,Jun
2,AU-2015-3,12/5/2015,12/13/2015,8.0,First Class,Auto & Accessories,Car Body Covers,117.00,5,0.01,...,3.10,Critical,EN-003,Moon Weien,Consumer,Porirua,Wellington,New Zealand,Oceania,Dec
3,AU-2015-4,5/9/2015,5/16/2015,7.0,First Class,Auto & Accessories,Car & Bike Care,118.00,2,0.05,...,2.60,High,AN-004,Sanchez Bergman,Corporate,Kabul,Kabul,Afghanistan,Central Asia,May
4,AU-2015-5,7/9/2015,7/18/2015,9.0,First Class,Auto & Accessories,Tyre,250.00,1,0.04,...,16.00,Critical,ON-005,Rowe Jackson,Corporate,Townsville,Queensland,Australia,Oceania,Jul


In [3]:
df = pd.get_dummies(df, columns=['Ship Mode', 'Product Category', 'Product', 'Order Priority','Segment', 'City', 'State', 'Country', 'Region', 'Months'])
df.head()

,Order ID,Order Date,Ship Date,Aging,Sales,Quantity,Discount,Profit,Shipping Cost,Customer ID,...,Months_Dec,Months_Feb,Months_Jan,Months_Jul,Months_Jun,Months_Mar,Months_May,Months_Nov,Months_Oct,Months_Sep
0,AU-2015-1,11/9/2015,11/17/2015,8.0,140.00,2,0.05,46.0,4.60,LS-001,...,0,0,0,0,0,0,0,1,0,0
1,AU-2015-2,6/30/2015,7/2/2015,2.0,211.00,3,0.03,112.0,11.20,IZ-002,...,0,0,0,0,1,0,0,0,0,0
2,AU-2015-3,12/5/2015,12/13/2015,8.0,117.00,5,0.01,31.2,3.10,EN-003,...,1,0,0,0,0,0,0,0,0,0
3,AU-2015-4,5/9/2015,5/16/2015,7.0,118.00,2,0.05,26.2,2.60,AN-004,...,0,0,0,0,0,0,1,0,0,0
4,AU-2015-5,7/9/2015,7/18/2015,9.0,250.00,1,0.04,160.0,16.00,ON-005,...,0,0,0,1,0,0,0,0,0,0


In [4]:
df2 = df.drop(['Order ID', 'Order Date', 'Ship Date', 'Customer ID', 'Customer Name'], axis = 1)
df2.head()

,Aging,Sales,Quantity,Discount,Profit,Shipping Cost,Ship Mode_45788,Ship Mode_First Class,Ship Mode_Same Day,Ship Mode_Second Class,...,Months_Dec,Months_Feb,Months_Jan,Months_Jul,Months_Jun,Months_Mar,Months_May,Months_Nov,Months_Oct,Months_Sep
0,8.0,140.00,2,0.05,46.0,4.60,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2.0,211.00,3,0.03,112.0,11.20,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,8.0,117.00,5,0.01,31.2,3.10,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
3,7.0,118.00,2,0.05,26.2,2.60,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,9.0,250.00,1,0.04,160.0,16.00,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import SGDClassifier

X, y = train_test_split(df2, test_size=0.2, random_state=42)

clf = SGDClassifier()

kf = KFold(n_splits=2)
kf_splits = kf.split(X)

train_index, test_index = next(kf_splits)
# partial-fit with training data. On the first call
# the classes must be provided

clf.partial_fit(X[train_index], y[test_index])

# re-training on new data
train_index, test_index = next(kf_splits)
clf.partial_fit(X[train_index], y[test_index])

KeyError: "None of [Int64Index([20516, 20517, 20518, 20519, 20520, 20521, 20522, 20523, 20524,\n            20525,\n            ...\n            41022, 41023, 41024, 41025, 41026, 41027, 41028, 41029, 41030,\n            41031],\n           dtype='int64', length=20516)] are in the [columns]"

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import SGDClassifier
train_x, test_x, train_y, test_y = train_test_split(df2.drop('Profit', axis=1), df['Profit'], test_size=0.2, random_state=42)

In [ ]:
clf = SGDClassifier()

kf = KFold(n_splits=2)
kf_splits = kf.split(df2.drop('Profit'))

train_x, test_y = next(kf_splits)
# partial-fit with training data. On the first call
# the classes must be provided
clf.partial_fit(df2.drop('Profit')[train_y],  df['Profit'][train_x, classes=np.unique(df['Profit']))

train_x, test_x = next(kf_splits)
clf.partial_fit(X[train_x], y[train_x])